# 导出外部动态链接库
- cdll
- windll:(windows)
- oledll:(windows)

- linux 下必须使用 包含 文件扩展名的文件名来导入共享库。因此不能简单使用对象属性的方式来导入库、


In [3]:
# win
from ctypes import *
print(windll.kernel32)
print(windll.user32)


# linux
# cdll.LoadLibrary("libc.so.6")
# libc = CDLL("libc.so.6")
# print(libc)

<WinDLL 'kernel32', handle 75330000 at 0x2fabfd0>
<WinDLL 'user32', handle 769b0000 at 0x2f87eb0>


# 操作动态库

## xxxA和xxxW的区别：
Win32系统的动态库，比如 kernel32 和 user32，
通常会同时导出同一个函数的 ANSI 版本和 UNICODE 版本。UNICODE
 版本通常会在名字最后以 W 结尾，而 ANSI 版本的则以 A 结尾

In [4]:
print(windll.kernel32.GetModuleHandleA)


# dlls的导出的函数名不符合 Python 的标识符规范，比如 "??2@YAPAXI@Z"。->getattr() 方法来获得该函数。
getattr(cdll.msvcrt, "??2@YAPAXI@Z")

# 有些dll没有函数名，可通过索引
cdll.kernel32[1]

## error
print(windll.kernel32.myfunction)

AttributeError: function 'myfunction' not found

# 数据类型
-   None，整型，字节对象和（UNICODE）字符串是仅有的可以直接作为函数参数使用的四种Python本地数据类型。
    None` 作为C的空指针 (NULL)，字节和字符串类型作为一个指向其保存数据的内存块指针 （char * 或 wchar_t *)。
    Python 的整型则作为平台默认的C的 int 类型，他们的数值被截断以适应C类型的整型长度。

类型[链接]("https://docs.python.org/zh-cn/3.7/library/ctypes.html#ctypes.c_char")

In [ ]:
i=c_int(10)
print(i)
print(i.value)



- 给指针类型的对象 c_char_p, c_wchar_p 和 c_void_p 等赋值时，
将改变它们所指向的内存地址，而不是它们所指向的内存区域的内容


In [ ]:
s="hello werido"
c_point=c_wchar_p(s)
print(c_point) # 指向地址
print(c_point.value) # 地址的值
c_point.value="hello liu" # 这里是指向了一个新地址，内容为 "hello liu"
print(c_point) # 地址已经改变



#### create_string_buffer()
- 可以直接改变内存块的值
- 内存块内容可以通过 raw 属性存取，希望将它作为NUL结束的字符串，请使用 value 属性

In [ ]:
from ctypes import create_string_buffer
p = create_string_buffer(3)
print(sizeof(p),repr(p.raw))

p = create_string_buffer(b"hello werido")
print(sizeof(p),repr(p.raw))

# 改变p的内存块的值
p.value=b"hello liu"
print(sizeof(p),repr(p.raw)) # 地址值其实没有改变，值改变了

#### 调用函数
- 打印：printf = libc.printf
- 除了整数、字符串以及字节串之外，所有的 Python 类型都必须使用它们对应的 ctypes 类型包装，才能够被正确地转换为所需的C语言类型

In [ ]:
from ctypes import cdll
libc = cdll.msvcrt   # 微软的标准C库，都是有这个为标准的
printf =libc.printf
printf(b"Hello, %s\n", b"World!")
printf(b"An int %d, a double %f\n", 1234, c_double(3.14))

#### 自定义数据类型
- ctypes 会寻找 **_as_parameter_** 属性并使用它作为函数参数。当然，它必须是数字、字符串或者二进制字符串

In [ ]:
class MyCtypes:
    def __init__(self,value):
        self._as_parameter_=value

myctypes=MyCtypes(value=45)
printf(b"%d bottles of beer\n", myctypes)
# 打印输出：
#>>> 42 bottles of beer 打印1，这里在IDE里面不会显示
#>>> 19                 打印2

### 返回类型
#### 默认情况下都会假定函数返回C int 类型。其他返回类型可以通过设置函数对象的 restype 属性来指定

In [ ]:
#strchr 函数，这个函数接收一个字符串指针以及一个字符作为参数 返回另一个字符串指针。
strchr =libc.strchr
strchr(b"abcdef", ord("d"))

strchr.restype = c_char_p
# 自定义类型strchr返回一个c_char_p


## 传递指针作为参数
- byref()
- pointer()

In [ ]:
i =c_int()
f = c_float()
s =create_string_buffer(b"hello werido")

print(i.value,f.value,repr(s.value))

# 往三个地址中写入新的值
libc.sscanf(b"1 3.14 Hellowerido", b"%d %f %s",byref(i), byref(f), s)
print(i.value, f.value, repr(s.value))


######################## pointer ##############################
i =c_int(42)
pi =pointer(i)
print(pi.contents)
 # contents 属性，它存储了指针指向的真实对象，如上面的 i 对象

# 改变PI的指向
b =c_int(99)
pi.contents=b
print(pi.contents)

## 指针访问
print(pi[0])
print(pi.contents.value)

#### 结构体和联合
- 继承自 Structure or Union
- 子类必须定义 **_fields_** 属性。 **_fields_** 是一个二元组列表，二元组中包含 field name 和 field type 。
- type 字段必须是一个 ctypes 类型，比如 c_int，或者其他 ctypes 类型: 结构体、联合、数组、指针。

In [7]:
from ctypes import *
class PointStruct(Structure):
    _fields_=[
        ("x",c_int), # 结构体的类型必须为ctypes
        ("y",c_int)
    ]
point= PointStruct(10,20)
print(point.x, point.y)

class Rect(Structure):
    _fields_=[
        ("upperleft",PointStruct),
        ("lowerright",PointStruct)
    ]
point1=PointStruct(1,2)
point2=PointStruct(3,4)
rc =Rect(point1) # 只对upperleft初始化
print(rc.upperleft.x, rc.upperleft.y)
print(rc.lowerright.x, rc.lowerright.y)
rc =Rect(point1,point2) #对upperleft lowerright 都初始化
print(rc.upperleft.x, rc.upperleft.y)
print(rc.lowerright.x, rc.lowerright.y)


##################### 结构体指定字段的长度 #######################
class Intwithlen(Structure):
    _fields_=[
        ("int_16",c_int,16),
        ("second_16",c_int,16)
    ]
print(Intwithlen.int_16)
print(Intwithlen.second_16)
a = 11111111111111111111111111  # 溢出 16位长度为0到6553
b = 22222222222  # 溢出   16位长度为0到6553
c = 123
d = 456
INT = Intwithlen(a,b)
INT2 = Intwithlen(c,d)
print(INT.int_16)
print(INT.second_16)
print(INT2.int_16)
print(INT2.second_16)

10 20
1 2
0 0
1 2
3 4
<Field type=c_long, ofs=0:0, bits=16>
<Field type=c_long, ofs=0:16, bits=16>
29127
13198
123
456


#### 数组
- 一个类型 * 一个正数: TenPointsArrayType = PointStruct *4

In [6]:
class ArrayItem(Structure):
    _fields_ = ("x",c_int),("y",c_int)

class MyStruct(Structure):
    _fields_=[
        ("a",c_int),
        ("b",c_float),
        ("point_array",PointStruct*4)
    ]
print(len(MyStruct().point_array))

NameError: name 'PointStruct' is not defined

#### 强类型转换
- ctypes 具有严格的类型检查。这代表着,如果在函数argtypes中或者结构体定义成员中有 POINTER(c_int)类型，只有相同类型的实例才会被接受。
- 例外：可以传递兼容的数组实例给指针类型。所以，对于 POINTER(c_int) ，ctypes 也可以接受 c_int 类型的数组
- cast()cast() 函数可以将一个指针实例强制转换为另一种 ctypes 类型。 cast()接收两个参数，
一个 ctypes 指针对象或者可以被转换为指针的其他类型对象，和一个 ctypes 指针类型。返回第二个类型的一个实例，
该返回实例和第一个参数指向同一片内存空间。 cast((c_byte * 4)(), POINTER(c_int))：第一个数数组类型，数组元素
为c_byte类型，第二个为c_int的指针类型，将第一个指向的内存的类型转换为c_int类型

In [9]:
class Bar(Structure):
    _fields_=[
        ("count",c_int),
        ("values",POINTER(c_int))
    ]
bar =Bar()
bar.values =(c_int*3)(1,2,3) # values 原本指向一个int类型的指针，现在变成改为一个数组
bar.count=3
for i in range(bar.count):
    print(bar.values[i])

##################### cast() ###########################
a=(c_byte*4)() # 数组默认为0,非比特类型.所以这里肯定会报错

bar=Bar()
# cast()将c_byte转换为c_int类型，这里注意必须为指针类型
bar.values =cast((c_byte*4)(),POINTER(c_int))#
print(bar.values[0])

1
2
3
0


#### 不完整类型
- 不完整类型,即还没有定义成员的结构体、联合或者数组。
- 在C中，它们通常用于前置声明，然后在后面定义

In [21]:
# C语言版
# struct cell
# struct cell{
#     char *name;
#     struct cell *next;
# }
# ctypes版本
## 错误版本
# class cell(Structure):
#     _fields_=[
#         ("name",c_char_p),
#         ("next",POINTER(cell))  # 这里错误，CELL未定义
#     ]

## 正确版本
class cell(Structure):
    pass
cell._fields_=[
    ("name",c_char_p),
    ("next",POINTER(cell))
] # 链式节点

c1=cell()
c2=cell()
## c语言没有字符串的概念，必须转位字节
c1.name=bytes("head/头部",'utf-8')
c2.name=bytes("tail/尾部",'utf-8')
c1.next=pointer(c2)
c2.next=pointer(c1)
p=c1
for i in  range(8):
    print(p.name.decode())
    p =p.next[0]


head/头部 <class 'bytes'>
tail/尾部 <class 'bytes'>
head/头部 <class 'bytes'>
tail/尾部 <class 'bytes'>
head/头部 <class 'bytes'>
tail/尾部 <class 'bytes'>
head/头部 <class 'bytes'>
tail/尾部 <class 'bytes'>


#### 回调函数
- ctypes允许创建一个指向Python 可调用对象的 C 函数
- 必须为回调函数创建一个类.这个类知道调用约定.包括返回值类型以及函数接收的参数类型及个数
- CFUNCTYPE() 工厂函数使用 cdecl 调用约定创建回调函数类型。
在 Windows 上，WINFUNCTYPE() 工厂函数使用 stdcall 调用约定为回调函数创建类型
- cdecl:
- stdcall:
- 所有的工厂函数用返回值类型作为第一个参数，回掉函数的参数类型作为剩余参数。

In [22]:
IntArray = c_int*5
ia = IntArray(5,1,7,33,99)
qsort = libc.qsort #  qsort() 将用来给整数数组排序
# qsort() 必须接收的参数，一个指向待排序数据的指针，元素个数，每个元素的大小，
# 以及一个指向排序函数的指针，即回调函数。
# 回调：回调函数接收两个元素的指针，如果第一个元素小于第二个，则返回一个负整数，如果相等则返回0，否则返回一个正整数

# 根据py_cmp_func来定,第一个为返回值类型，后面为参数类型,指向C_INT的指针
CMPFUNC=CFUNCTYPE(c_int,POINTER(c_int),POINTER(c_int)) #
def py_cmp_func(a,b):
    print("py_cmp_func",a[0],b[0])
    return a[0]-b[0]

cmp_func=CMPFUNC(py_cmp_func)

qsort(ia,len(ia),sizeof(c_int),cmp_func)

## 另外一种写法
@CFUNCTYPE(c_int,POINTER(c_int),POINTER(c_int))
def py_cmp_func(a,b):
    print("py_cmp_func",a[0],b[0])
    return a[0]-b[0]

qsort(ia,len(ia),sizeof(c_int),cmp_func)

NameError: name 'libc' is not defined

#### 同一空间赋值注意事项

In [24]:
class POINT(Structure):
    _fields_=("x", c_int), ("y", c_int)

class RECT(Structure):
    _fields_ = ("a", POINT), ("b", POINT)

p1=POINT(1,2)
p2=POINT(3,4)
rc=RECT(p1,p2)
print(rc.a.x,rc.a.y,rc.b.x,rc.b.y)

rc.a,rc.b = rc.b,rc.a
print(rc.a.x,rc.a.y,rc.b.x,rc.b.y)
# 这里打印出来并不是 34 12 而是 3434
# 赋值过程如下：
# temp0,temp1=rc.b,rc.a
# rc.a=temp0
# rc.b=temp1
# 注意 temp0 和 temp1 对象始终引用了对象 rc 的内容。
# 然后执行 rc.a = temp0 会把 temp0 的内容拷贝到 rc 的空间。这也改变了 temp1 的内容

# 例子2：
s = c_char_p() #实例化的对象只能将其值设置为 bytes 或者整数。
s.value=b"test"
print(s.value is s.value)
# False

1 2 3 4
3 4 3 4


#### 寻找动态链接库
- ctypes.util.find_library(name)
- ctypes.CDLL(name, mode=DEFAULT_MODE, handle=None, use_errno=False, use_last_error=False)：此类的实例即已加载的动态链接库.库中的函数使用标准 C 调用约定，并假定返回 int 。
- ctypes.OleDLL(name, mode=DEFAULT_MODE, handle=None, use_errno=False, use_last_error=False):仅 Windows,此类的实例即加载好的动态链接库，其中的函数使用 stdcall 调用约定，
并且假定返回 windows 指定的 HRESULT 返回码。
- ctypes.WinDLL(name, mode=DEFAULT_MODE, handle=None, use_errno=False, use_last_error=False):仅 Windows,此类的实例即加载好的动态链接库，其中的函数使用 stdcall 调用约定，并假定默认返回 int
##### 参数
- name:通过使用至少一个参数（共享库的路径名）调用它们，可以实例化所有这些类。也可以传入一个已加载的动态链接库作为 handler 参数，
其他情况会调用系统底层的 dlopen 或 LoadLibrary 函数将库加载到进程，并获取其句柄
- mode 可以指定库加载方式。在Windows上，会忽略mode,在posix系统上，总是会加上 RTLD_NOW ，且无法配置
- use_errno 参数如果设置为 true，可以启用ctypes的机制，通过一种安全的方法获取系统的 errno 错误码。
ctypes 维护了一个线程局部变量，它是系统 errno 的一份拷贝；如果调用了使用 use_errno=True 创建的外部函数，
errno 的值会与 ctypes 自己拷贝的那一份进行交换，函数执行完后立即再交换一次
- use_last_error 参数如果设置为 true，可以在 Windows 上启用相同的策略，它是通过 Windows API 函数 GetLastError()  和 SetLastError() 管理的。
ctypes.get_last_error() 和 ctypes.set_last_error() 可用于获取和设置 ctypes 自己维护的 windows 错误码拷贝

##### mode
- ctypes.RTLD_GLOBAL:用于 mode 参数的标识值。在此标识不可用的系统上，它被定义为整数0
- ctypes.RTLD_LOCAL:
- ctypes.DEFAULT_MODE: